# 建立您的深度神经网络：循序渐进

欢迎来到第4周的作业（第1部分，共2部分）！您之前已经训练了2层神经网络（具有一个隐藏层）。本周，您将构建一个深度神经网络，其中包含任意数量的层！

- 在此笔记本中，您将实现构建深度神经网络所需的所有功能。
- 在下一个作业中，您将使用这些功能来构建用于图像分类的深度神经网络。

**完成此任务后，您将能够：**
- 使用ReLU等非线性单位来改善模型
- 建立更深的神经网络（具有1个以上的隐藏层）
- 实施易于使用的神经网络类

**符号**：
- 上标$[l]$表示与$l^{th}$层相关的数量。
    - 示例：$a^{[L]}$是$L^{th}$ 层激活。 $W^{[L]}$和$b^{[L]}$是$L^{th}$层参数。
- 上标$(i)$表示与$i^{th}$样本相关的数量。
    - 示例：$x^{(i)}$是$i^{th}$ 训练样本。
- 下标$i$表示向量的$i^{th}$条目。
    - 示例：$a^{[l]}_i$表示$l^{th}$层激活的$i^{th}$条目）。

让我们开始吧！

## 1-软件包

首先，让我们导入在此分配过程中需要的所有软件包。
- [numpy](www.numpy.org)是使用Python进行科学计算的主要软件包。
- [matplotlib](http://matplotlib.org)是一个用于在Python中绘制图形的库。
- dnn_utils为此笔记本提供了一些必要的功能。
- testCases提供了一些测试用例以评估功能的正确性
- np.random.seed（1）用于使所有随机函数调用保持一致。 这将帮助我们为您的作品评分。 请不要改变种子。

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots 设置plots的默认大小
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2-作业大纲

为了构建您的神经网络，您将实现几个“辅助函数”。这些辅助函数将在下一个任务中使用，以构建一个两层神经网络和一个L层神经网络。您将实现的每个小辅助函数都有详细的说明，这些步骤将指导您完成必要的步骤。这是此作业的概述，您将：

- 初始化两层网络和$L$层神经网络的参数。
- 实施前向传播模块（在下图中以紫色显示）。
     - 完成图层前向传播步骤的LINEAR部分（得到$Z^{[l]}$）。
     - 我们为您提供ACTIVATION函数（relu / Sigmoid）。
     - 将前两个步骤合并为新的[LINEAR->ACTIVATION] 前向函数。
     - 堆叠 [LINEAR->RELU] 正向函数L-1次（对于第1到L-1层），并在末尾添加[LINEAR->SIGMOID]（对于最后一层$L$）。这为您提供了一个新的L_model_forward函数。
- 计算损失。
- 实施反向传播模块（在下图中以红色表示）。
    - 完成图层向后传播步骤的LINEAR部分。
    - 我们为您提供ACTIVATE函数的导数（relu_backward / sigmoid_backward）
    - 将前两个步骤合并为新的[LINEAR-> ACTIVATION]后向函数。
    - 将[LINEAR-> RELU]后向堆叠L-1次，并在新的L_model_backward函数中向后添加[LINEAR-> SIGMOID]
- 最后更新参数。

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center> **Figure 1**</center></caption><br>


**请注意**，对于每个前向函数，都有一个相应的反向函数。这就是为什么在前向模块的每一步都将一些值存储在缓存中的原因。缓存的值可用于计算梯度。然后，在反向传播模块中，您将使用缓存来计算梯度。该作业将向您确切说明如何执行这些步骤。

## 3-初始化

您将编写两个辅助函数，这些函数将初始化模型的参数。 第一个函数将用于初始化两层模型的参数。 第二个步骤将把初始化过程推广到 $L$层。

### 3.1-2层神经网络

**练习**：创建并初始化2层神经网络的参数。

**说明**：
- 模型的结构为：*LINEAR -> RELU -> LINEAR -> SIGMOID*。
- 对权重矩阵使用随机初始化。 使用正确形状的`np.random.randn（shape）* 0.01`。
- 对偏差使用零初始化。 使用`np.zeros（shape）`。

In [5]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [6]:
parameters = initialize_parameters(2,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[0.]]


**预期输出**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td> [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]] </td> 
  </tr>

  <tr>
    <td> **b1**</td>
    <td>[[ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.00865408 -0.02301539]]</td>
  </tr>
  
  <tr>
    <td> **b2** </td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 3.2-L层神经网络

更深的L层神经网络的初始化更加复杂，因为存在更多的权重矩阵和偏差向量。完成`initialize_parameters_deep`后，应确保各层之间的尺寸匹配。回想一下，$n^{[l]}$是层$l$中的单位数。因此，例如，如果我们输入的$X$ 的大小为$(12288, 209)$（以$m=209$为例），则：

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td>   
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    
   <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    
    
   <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>

</table>

请记住，当我们在python中计算$W X + b$ 时，它将执行广播。例如，如果：

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$


那么$WX + b$将是：

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**练习**：为L层神经网络实现初始化。

**说明**：
- 模型的结构为*[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*.。也就是说，它具有使用ReLU激活功能的 $L-1$层，然后是具有S型激活功能的输出层。
- 对权重矩阵使用随机初始化。使用`np.random.rand（shape）* 0.01`。
- 将零初始化用于偏差。使用`np.zeros（shape）`。
- 我们将$n^{[l]}$（即不同层中的单位数）存储在变量`layer_dims`中。例如，上周“平面数据分类模型”的`layer_dims`为[2,4,1]：有两个输入，一个隐藏层包含4个隐藏单元，一个输出层包含1个输出单元。因此，意味着` W1`的形状为（4,2），`b1`的形状为（4,1），` W2`的形状为（1,4），而` b2`的形状为（1,1）。现在，您将把它推广到$L$层！
- 这是$L=1$（一层神经网络）的实现。它会激发您实施一般情况（L层神经网络）。
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [9]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [10]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>**b1** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2** </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>**b2** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4-前向传播模块

### 4.1-线性正向
现在，您已经初始化了参数，接下来将执行正向传播模块。 您将首先实现一些基本函数，稍后将在实现模型时使用这些函数。 您将按以下顺序完成三个函数：
 
- 线性
- 线性->激活，其中激活将是ReLU或Sigmoid。
- [线性-> RELU]  $\times$（L-1）->线性-> SIGMOID（整个模型）

线性正向模块（在所有示例中均进行矢量化）计算以下方程式：

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$


其中$A^{[0]} = X$。

**练习**：建立正向传播的线性部分。

**提醒**：
该单元的数学表示为$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$。 您可能还会发现`np.dot（）`有用。 如果尺寸不匹配，则可以打印 `W.shape`。

In [11]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W, A) + b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [12]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**预期输出**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4.2-线性激活正向

在此笔记本中，您将使用两个激活功能：

- **Sigmoid**： $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$。 我们为您提供了`sigmoid`功能。 该函数返回 **两项**：激活值“`a`”和包含“`Z`”的“`cache`”（这是我们将馈入到相应的后向函数的内容）。 要使用它，您可以调用：
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**：ReLu的数学公式为$A = RELU(Z) = max(0, Z)$。 我们为您提供了`relu`函数。 该函数返回**两项**：激活值“`A`”和包含“`Z`”的“`cache`”（这是我们将馈入相应的后向函数的内容）。 要使用它，您可以调用：
``` python
A, activation_cache = relu(Z)
```

为了更加方便，您将把两个功能（线性和激活）组合为一个功能（LINEAR-> ACTIVATION）。 因此，您将实现执行LINEAR前进步骤，然后执行ACTIVATION前进步骤的功能。

**练习**：实现*LINEAR->ACTIVATION*层的正向传播。 数学关系为：$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$，其中激活“g”可以是sigmoid()或relu()。 使用linear_forward()和正确的激活函数。

In [13]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [14]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**预期输出**:
       
<table style="width:35%">
  <tr>
    <td> **With sigmoid: A ** </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> **With ReLU: A ** </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>


**注意**：在深度学习中，“ [LINEAR-> ACTIVATION]”计算被视为神经网络中的单个层，而不是两个层。

### d）L层模型

为了在实现$L$层神经网络时提供更多便利，您将需要一个函数来复制前一个函数（使用RELU的`linear_activation_forward`）$L-1$ 次，然后再使用一个带有SIGMOID的`linear_activation_forward`。

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **Figure 2** : *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model</center></caption><br>

**练习**：实现上述模型的正向传播。

**说明**：在下面的代码中，变量`AL`将表示$A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$。 （有时也称为`Yhat`，即$\hat{Y}$。）

**提示**：
- 使用您先前编写的函数
- 使用for循环复制[LINEAR-> RELU]（L-1）次
- 不要忘记在“cache”列表中跟踪缓存。 要将新值`c`添加到`list`中，可以使用`list.append（c）`。

In [19]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], "relu")

        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")

    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [20]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 0


<table style="width:40%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.17007265  0.2524272 ]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 2</td> 
  </tr>
</table>

太棒了！现在，您有了一个完整的正向传播，它接受输入X并输出一个包含预测的行向量$A^{[L]}$。 它还将所有中间值记录在“缓存”中。 使用$A^{[L]}$，您可以计算预测成本。

## 5-成本函数

现在，您将实现正向和反向传播。 您需要计算成本，因为您要检查模型是否在实际学习中。

**练习**：使用以下公式计算交叉熵成本$J$：$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

In [40]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = -1 / m * np.sum(Y * np.log(AL) + (1-Y)*np.log(1 - AL), axis = 1, keepdims = True)
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [41]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


**预期输出**:

<table>
    <tr>
    <td>**cost** </td>
    <td> 0.41493159961539694</td> 
    </tr>
</table>

## 6-向后传播模块

就像正向传播一样，您将实现辅助函数以进行反向传播。请记住，反向传播用于计算损耗函数相对于参数的梯度。

**提醒**：
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center> **Figure 3** : Forward and Backward propagation for *LINEAR->RELU->LINEAR->SIGMOID* <br> *The purple blocks represent the forward propagation, and the red blocks represent the backward propagation.*  </center></caption>

<!--
对于那些精通微积分的人（您不必进行此分配），可以使用微积分的链式法则来推导2层网络中损失 $\mathcal{L}$相对于 $z^{[1]}$的导数：
$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$


为了计算梯度$dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$，请使用上一个链式法则，然后执行$dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$。在反向传播期间，在每个步骤中，您都将当前梯度乘以对应于特定图层的梯度，以获得所需的梯度。

等效地，为了计算梯度 $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$，您可以使用上一个链规则，然后执行$db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$。

这就是为什么我们谈论**反向传播**。
!-->

现在，类似于正向传播，您将分三个步骤构建反向传播：
- 线性向后
- 线性->向后激活，其中激活可计算ReLU或S型激活的导数
- [线性-> RELU]  $\times$（L-1）->线性-> SIGMOID向后（整个模型）

### 6.1-线性后向

对于层$l$，线性部分为：$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ （之后是激活）。

假设您已经计算出导数$dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$。 您想获得$(dW^{[l]}, db^{[l]} dA^{[l-1]})$。

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>


使用输入$dZ^{[l]}$计算三个输出$(dW^{[l]}, db^{[l]}, dA^{[l]})$。以下是所需公式：
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

**练习**：使用上面的3个公式来实现linear_backward（）。

In [44]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    ### START CODE HERE ### (≈ 3 lines of code)
    dW = 1 / m * np.dot(dZ, A_prev.T)
    db = 1 / m *np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [45]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


**预期输出**: 

<table style="width:90%">
  <tr>
    <td> **dA_prev** </td>
    <td > [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]] </td> 
  </tr> 
    <tr>
        <td> **dW** </td>
        <td > [[-0.10076895  1.40685096  1.64992505]] </td> 
    </tr> 
    <tr>
        <td> **db** </td>
        <td> [[ 0.50629448]] </td> 
    </tr> 
    
</table>



### 6.2-后向线性激活

接下来，您将创建一个合并两个辅助函数的函数： **`linear_backward`** 和激活的向后步骤 **`linear_activation_backward`**。

为了帮助您实现`linear_activation_backward`，我们提供了两个后向函数：
- **`sigmoid_backward`**：实现SIGMOID单元的后向传播。 您可以按以下方式调用它：

```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward` **：实现RELU单元的后向传播。 您可以按以下方式调用它：

```python
dZ = relu_backward(dA, activation_cache)
```

如果$g(.)$是激活函数，
`sigmoid_backward`和`relu_backward`计算
$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$

**练习**：对*LINEAR-> ACTIVATION* 层进行反向传播。

In [48]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [49]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**sigmoid函数的预期输出:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 

  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>



**relu函数的预期输出**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 

  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>



### 6.3-后向L型

现在，您将为整个网络实现后向函数。回想一下，当您实现`L_model_forward`函数时，在每次迭代中，您都存储了一个包含（X，W，b和z）的缓存。在反向传播模块中，您将使用这些变量来计算梯度。因此，在`L_model_backward`函数中，您将从 $L$层开始向后遍历所有隐藏层。在每个步骤上，您都将使用图层$l$ 的缓存值反向传播到图层$l$ 。下面的图5显示了反向传递。


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **Figure 5** : Backward pass  </center></caption>

**初始化反向传播**：
为了通过这个网络反向传播，我们知道输出是
$A^{[L]} = \sigma(Z^{[L]})$。因此，您的代码需要计算`dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$。
为此，请使用以下公式（使用不需要深入知识的微积分得出）：
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```

然后，您可以使用此激活后的梯度`dAL`继续向后移动。如图5所示，您现在可以将`dAL`输入到您实现的LINEAR-> SIGMOID向后函数中（它将使用L_model_forward函数存储的缓存值）。之后，您将必须使用`for`循环，使用LINEAR-> RELU向后函数迭代所有其他层。您应该将每个dA，dW和db存储在grads词典中。为此，请使用以下公式：

 $$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

例如，对于 l = 3 ，这会将 $dW^{[l]}$ 存储在`grads["dW3"]`中。

**练习**：对 *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* 模型实施反向传播。

In [55]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ...
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ...
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = - (np.divide(Y ,AL) - np.divide(1 - Y, 1 -AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[L - 1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    ### END CODE HERE ###
    
    for l in reversed(range(L - 1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+2)], current_cache, "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [56]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


**预期输出**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]] </td> 

  </tr> 
</table>



### 6.4-更新参数

在本节中，您将使用梯度下降来更新模型的参数：

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$


其中 $\alpha$是学习率。 计算更新的参数后，将它们存储在参数字典中。

**练习**：实现`update_parameters（）`以使用梯度下降来更新参数。

**说明**：
在每个$W^{[l]}$ and $b^{[l]}$ 上使用梯度下降来更新参数，其中$l = 1, 2, ..., L$。

In [59]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l+1)]
    ### END CODE HERE ###
        
    return parameters

In [60]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**预期输出**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>



## 7-结论

恭喜您实现了构建深度神经网络所需的所有功能！

我们知道这是一项艰巨的任务，但今后它只会变得更好。 下一部分的工作比较容易。

在下一个作业中，您将所有这些放在一起以构建两个模型：
- 两层神经网络
- L层神经网络

实际上，您将使用这些模型对猫图像和非猫图像进行分类！